<a href="https://colab.research.google.com/github/kullawattana/thesis_2020_spacy_colab/blob/master/21_main_find_SVO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys

# NLP Processing
import spacy
from spacy.matcher import Matcher
from spacy.symbols import nsubj, VERB

SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
OBJECTS = ["dobj", "dative", "attr", "oprd"]

#---------------------------------Step 1 Import Data && step 2 custom sentence boundary && Step 3 custom pipeline---------------------------------
def customSentenceBoundaries(doc):
    for token in doc[:-1]:
        if token.text == "otherwise":
            doc[token.i+1].is_sent_start = True
        if token.text == "and":
            doc[token.i].is_sent_start = True
        if token.text == "...":
            doc[token.i].is_sent_start = True
    return doc

#---------------------------------step 4 find SVO---------------------------------
def getSubsFromConjunctions(subs):
    moreSubs = []
    for sub in subs:
        # rights is a generator
        rights = list(sub.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreSubs.extend([tok for tok in rights if tok.dep_ in SUBJECTS or tok.pos_ == "NOUN"])
            if len(moreSubs) > 0:
                moreSubs.extend(getSubsFromConjunctions(moreSubs))
    return moreSubs

def findSubs(tok):
    head = tok.head
    while head.pos_ != "VERB" and head.pos_ != "NOUN" and head.head != head:
        head = head.head
    if head.pos_ == "VERB":
        subs = [tok for tok in head.lefts if tok.dep_ == "SUB"]
        if len(subs) > 0:
            verbNegated = isNegated(head)
            subs.extend(getSubsFromConjunctions(subs))
            return subs, verbNegated
        elif head.head != head:
            return findSubs(head)
    elif head.pos_ == "NOUN":
        return [head], isNegated(tok)
    return [], False

def getAllSubs(v):
    verbNegated = isNegated(v)
    subs = [tok for tok in v.lefts if tok.dep_ in SUBJECTS and tok.pos_ != "DET"]
    if len(subs) > 0:
        subs.extend(getSubsFromConjunctions(subs))
    else:
        foundSubs, verbNegated = findSubs(v)
        subs.extend(foundSubs)
    return subs, verbNegated

def isNegated(tok):
    negations = {"no", "not", "n't", "never", "none"}
    for dep in list(tok.lefts) + list(tok.rights):
        if dep.lower_ in negations:
            return True
    return False

def findSVOs(tokens):
    svos = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]

    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        if len(subs) > 0:
            v, objs = getAllObjs(v)

            for sub in subs:
                for obj in objs:
                    objNegated = isNegated(obj)
                    svos.append((sub.lower_, "!" + v.lemma_ if verbNegated or objNegated else v.lemma_, obj.lower_))
    return svos

def getAllObjs(v):
    # rights is a generator
    rights = list(v.rights)
    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
    objs.extend(getObjsFromPrepositions(rights))

    potentialNewVerb, potentialNewObjs = getObjFromXComp(rights)
    if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
        objs.extend(potentialNewObjs)
        v = potentialNewVerb
    if len(objs) > 0:
        objs.extend(getObjsFromConjunctions(objs))
    return v, objs

def getObjsFromPrepositions(deps):
    objs = []
    for dep in deps:
        if dep.pos_ == "ADP" and dep.dep_ == "prep":
            objs.extend([tok for tok in dep.rights if tok.dep_  in OBJECTS or (tok.pos_ == "PRON" and tok.lower_ == "me")])
    return objs

def getObjFromXComp(deps):
    for dep in deps:
        if dep.pos_ == "VERB" and dep.dep_ == "xcomp":
            v = dep
            rights = list(v.rights)
            objs = [tok for tok in rights if tok.dep_ in OBJECTS]
            objs.extend(getObjsFromPrepositions(rights))
            if len(objs) > 0:
                return v, objs
    return None, None

def getObjsFromConjunctions(objs):
    moreObjs = []
    for obj in objs:
        # rights is a generator
        rights = list(obj.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreObjs.extend([tok for tok in rights if tok.dep_ in OBJECTS or tok.pos_ == "NOUN"])
            if len(moreObjs) > 0:
                moreObjs.extend(getObjsFromConjunctions(moreObjs))
    return moreObjs

def printDeps(toks):
    for tok in toks:
        print(tok.orth_, tok.dep_, tok.pos_, tok.head.orth_, [t.orth_ for t in tok.lefts], [t.orth_ for t in tok.rights])

#---------------------------------step 5 match pattern-----------------------------------
def match_pattern(text, patterns):
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)
    matcher.add('Boots', None, patterns)

    doc = nlp(text)
    matches = matcher(doc)

    for match in matches:
        match_id, start, end = match
        span = doc[start:end]
        print(match, span.text)

def check_subject_pattern(text):
    #find subject
    patterns = [
        {'POS': 'DET'},
        {'POS': 'NOUN'}
    ]
    match_pattern(text, patterns)

def check_subject_propn_pattern(text):
    #find subject
    patterns = [
        {'POS': 'DET'},
        {'POS': 'PROPN'}
    ]
    match_pattern(text, patterns)

def check_object_pattern(text):
    #find object
    patterns = [
        {'POS': 'ADJ'},
        {'POS': 'NOUN'}
    ]
    match_pattern(text, patterns)

def check_if_then_pattern(text):
    # IF-THEN, ...
    patterns = [
        {'POS': 'DET'},
        {'POS': 'NOUN'},
        {'POS': 'VERB'}
    ]
    match_pattern(text, patterns)

# Finding
def finding_subject_dependency(text):
    for possible_subject in doc:
        if possible_subject.dep == nsubj:
            check_subject_pattern(text)


if __name__ == "__main__":
    text = "this is a book."
    print(text)
    #--------------Step 1 Import----------------
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    print("Before:", [sent.text for sent in doc.sents])
    #--------------Step 2 Custom Boundary && Step 3 Adding Pipeline Custom Boundary----------------
    nlp.add_pipe(customSentenceBoundaries, before="parser")
    doc = nlp(text)
    print("After:", [sent.text for sent in doc.sents])
    #--------------Step 4 Find SVO-------------------
    sentences = list(doc.sents)
    for i, sent in enumerate(doc.sents):
        print(sent)         #print(i, "a", sent, type(sent))
        #step 4
        svos = findSVOs(sent)
        print(svos)
        #----------------Step 5 match pattern by Sentence----------------------
        finding_subject_dependency(text)

this is a book.
Before: ['this is a book.']
After: ['this is a book.']
this is a book.
[]
(18231591219755621867, 2, 4) a book
